In [1]:
import docreader
import re
import sys
from collections import defaultdict

In [2]:
SPLIT_RGX = re.compile(r'\w+', re.U)

def extract_words(text):
    words = re.findall(SPLIT_RGX, text)
    return map(lambda s: s.lower(), words)


In [3]:
myDict = defaultdict(lambda:[])
urls = []
# reader = DocumentStreamReader(parse_command_line().files)
reader = docreader.DocumentStreamReader(['dataset/lenta.ru_159b9f4b-972b-48b1-8ec3-44fbd6be33c4_01.gz'])
for doc in reader:
#     print "%s\t%d bytes" % (doc.url, len(doc.text))
    urls.append(doc.url)
    for word in extract_words(doc.text):
        myDict[word].append(len(urls) - 1)


In [4]:
def getUrls(words):
    result = []
    n = len(words)
    pointer = [0] * n
    for url1 in myDict[words[0]]:
        cnt = 1
        for i in range(1, n):
            curDict = myDict[words[i]]
            while pointer[i] < len(curDict) and curDict[pointer[i]] < url1:
                pointer[i] += 1
            if pointer[i] == len(curDict):
                break
            if curDict[pointer[i]] == url1:
                cnt += 1
        if cnt == n:
            result.append(urls[url1])
    return result

In [13]:
# назвала &самые &подверженные &коррупции
query = [q.strip() for q in raw_input().decode('cp1251').split("&")]
print query
result = getUrls(query)
print len(result)
for r in result:
    print r


назвала &самые &подверженные &коррупции
[u'\u043d\u0430\u0437\u0432\u0430\u043b\u0430', u'\u0441\u0430\u043c\u044b\u0435', u'\u043f\u043e\u0434\u0432\u0435\u0440\u0436\u0435\u043d\u043d\u044b\u0435', u'\u043a\u043e\u0440\u0440\u0443\u043f\u0446\u0438\u0438']
1
http://lenta.ru/news/2015/12/22/corruption_reiting/


In [25]:
def num_to_bytes(x):
    # x < 128
    res = ''
    for i in range(7):
        if x & (1 << i) > 0:
            res += '1'
        else:
            res += '0'
    res += '0'
    res = res[::-1]
    return res

def bytes_to_num(x):
    res = 0
    x = x[::-1]
    for i in range(len(x)):
        res += int(x[i]) * (1 << i)
    return res

def varbyte_encode(x):
    res = []
    while x > 0:
        cur = x % 128
        res.append(num_to_bytes(x))
        x /= 128
    res.reverse()
    res[0] = '1' + res[0][1:]    
    return ''.join(res)

def varbyte_decode(x):
    res = 0
    blocks = []
    for i in range(len(x) // 8):
        blocks.append(x[(i * 8):((i + 1) * 8)])
    blocks[0] = '0' + blocks[0][1:8]
    blocks.reverse()
    for i in range(len(blocks)):
        res += 128 ** i * bytes_to_num(blocks[i])
    return res

# print varbyte_encode(5)
# print varbyte_encode(128)
# print varbyte_encode(133)
# print varbyte_decode(varbyte_encode(5))
# print varbyte_decode(varbyte_encode(128))
# print varbyte_decode(varbyte_encode(133))

10000101
1000000100000000
1000000100000101
5
128
133
